In [9]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd

In [10]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [11]:
directions = ["left", "right", "back"]
all_coordinates = {}
for direction in directions:
    # 비디오 파일 이름들
    input_files = [f"D:\\Pitch\\sample_videos\\{i}_{direction}.mp4" for i in range(1,21)] # 1부터 20까지의 비디오 경로들

In [12]:
for direction in directions:
    # 비디오 파일 이름들
    input_files = [f"D:\\Pitch\\sample_videos\\{i}_{direction}.mp4" for i in range(1,21)] # 1부터 20까지의 비디오 경로들

    for input_file in input_files:
        if not os.path.exists(input_file):
            print(f"{input_file}이 존재하지 않습니다.")
            continue  # 파일이 존재하지 않으면 다음 파일로 넘어감

        # 비디오 파일의 경우 이것을 사용하세요:
        cap = cv2.VideoCapture(input_file)
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # 어깨 관절 좌표화 코드
        right_wrist_coordinates = [] # 우완투수

        with mp_pose.Pose(
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as pose:

            while cap.isOpened():
                success, image = cap.read()
                if not success:
                    print(f"{input_file}을 처리하는데 실패했습니다.")
                    break

                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = pose.process(image)

                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                image_height, image_width, _ = image.shape

                if results.pose_landmarks:
                    right_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST.value]
                    right_wrist_coordinates.append((right_wrist.x, right_wrist.y))

        cap.release()

        all_coordinates[input_file] = right_wrist_coordinates

        # 팔꿈치 좌표 출력
        for i, coord in enumerate(right_wrist_coordinates): # 왼손투수의 경우 left_shoulder_coordinates로 바꿔주세요.
            print(f"Frame {i+1}: x = {coord[0] * image_width}, y = {coord[1] * image_height}")

        # 좌표를 데이터프레임에 저장
        df = pd.DataFrame(right_wrist_coordinates, columns=["x", "y"]) # x_back, x_left, x_right, y_back, y_left, y_right 중 선택

        output_dir = "D:/Pitch/coordinates"
        output_filename = os.path.basename(os.path.splitext(input_file)[0]) + "_wrist_coordinates.csv"
        output_file_path = os.path.join(output_dir, output_filename)

        df.to_csv(output_file_path, index=False)

D:\Pitch\sample_videos\1_left.mp4을 처리하는데 실패했습니다.
Frame 1: x = 839.6827840805054, y = 945.864086151123
Frame 2: x = 840.1472353935242, y = 951.866455078125
Frame 3: x = 844.447546005249, y = 950.1334190368652
Frame 4: x = 845.9221386909485, y = 949.9508285522461
Frame 5: x = 842.9930377006531, y = 950.4935073852539
Frame 6: x = 843.2438349723816, y = 947.9433059692383
Frame 7: x = 843.2141590118408, y = 947.1362113952637
Frame 8: x = 837.8936648368835, y = 947.2056770324707
Frame 9: x = 838.0779647827148, y = 946.340446472168
Frame 10: x = 838.3346843719482, y = 942.1768569946289
Frame 11: x = 837.3576307296753, y = 940.8511734008789
Frame 12: x = 835.3706932067871, y = 941.0548210144043
Frame 13: x = 834.3730401992798, y = 941.5184783935547
Frame 14: x = 834.0098476409912, y = 942.1683883666992
Frame 15: x = 832.9543876647949, y = 942.3640823364258
Frame 16: x = 832.4710750579834, y = 942.491283416748
Frame 17: x = 831.3518857955933, y = 945.5899429321289
Frame 18: x = 830.786046981811

: 